In [1]:
import torch
import json
import torch
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import random
import math
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import clip

In [2]:
dataset_root = '/local/vondrick/ruoshi/objaverse/views_whole_sphere'

# dataset

In [3]:
class objaverse_sfm(Dataset):
    def __init__(self, root_dir, total_view, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        with open(os.path.join(root_dir, 'valid_paths.json')) as f:
            self.paths = json.load(f)import clip
        random.shuffle(self.paths)
        self.total_view = total_view
        self.train = train
        total_objects = len(self.paths)
        if train:
            self.paths = self.paths[:math.floor(total_objects / 100. * 99.)] # used first 99% as training|
        else:
            self.paths = self.paths[math.floor(total_objects / 100. * 99.):] # used last 1% as validation
        
    def __len__(self):
        return len(self.img_labels)

    def __len__(self):
        return len(self.paths)
        
    def cartesian_to_spherical(self, xyz):
        ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
        xy = xyz[:,0]**2 + xyz[:,1]**2
        z = np.sqrt(xy + xyz[:,2]**2)
        theta = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down
        #ptsnew[:,4] = np.arctan2(xyz[:,2], np.sqrt(xy)) # for elevation angle defined from XY-plane up
        azimuth = np.arctan2(xyz[:,1], xyz[:,0])
        return np.array([theta, azimuth, z])

    def get_T(self, target_RT, cond_RT):
        R, T = target_RT[:3, :3], target_RT[:, -1]
        T_target = -R.T @ T

        R, T = cond_RT[:3, :3], cond_RT[:, -1]
        T_cond = -R.T @ T

        theta_cond, azimuth_cond, z_cond = self.cartesian_to_spherical(T_cond[None, :])
        theta_target, azimuth_target, z_target = self.cartesian_to_spherical(T_target[None, :])
        
        d_theta = theta_target - theta_cond
        d_azimuth = (azimuth_target - azimuth_cond) % (2 * math.pi)
        d_z = z_target - z_cond
        
        d_T = torch.tensor([d_theta.item(), math.sin(d_azimuth.item()), math.cos(d_azimuth.item()), d_z.item()])
#         d_T = torch.tensor([d_theta.item(), d_azimuth.item(), d_z.item()])
        return d_T

    def load_im(self, path):
        '''
        replace background pixel with white in rendering
        '''
        img = plt.imread(path)
        img[img[:, :, -1] == 0.] = [1., 1., 1., 1.]
        img = Image.fromarray(np.uint8(img[:, :, :3] * 255.))
        return img

    def __getitem__(self, index):
        data = {}
        index_target, index_cond = random.sample(range(self.total_view), 2) # without replacement
        filename = os.path.join(self.root_dir, self.paths[index])

        # print(self.paths[index])

#         if self.return_paths:
#             data["path"] = str(filename)
            
        target_im = self.process_im(self.load_im(os.path.join(filename, '%03d.png' % index_target)))
        target_RT = np.load(os.path.join(filename, '%03d.npy' % index_target))
        cond_im = self.process_im(self.load_im(os.path.join(filename, '%03d.png' % index_cond)))
        cond_RT = np.load(os.path.join(filename, '%03d.npy' % index_cond))

        data["image_target"] = target_im
        data["image_cond"] = cond_im
        data["T"] = self.get_T(target_RT, cond_RT)
        return data
    
    def process_im(self, im):
        im = im.convert("RGB")
        im = self.transform(im)
        im = torchvision.transforms.functional.resize(im, 224)
        im = im * 2. - 1.
        return im

SyntaxError: invalid syntax (<ipython-input-3-9a652f559f71>, line 6)

In [4]:
dataset = objaverse_sfm(dataset_root, 4, train=True, transform = ToTensor())

In [5]:
dataset[0]['image_target'].shape

torch.Size([3, 224, 224])

In [30]:
train_dataloader = DataLoader(objaverse_sfm(dataset_root, 4, train=True, transform = ToTensor()),\
                              batch_size=16, shuffle=True, num_workers=4)
test_dataloader = DataLoader(objaverse_sfm(dataset_root, 4, train=False, transform = ToTensor()),\
                             batch_size=16, shuffle=False, num_workers=4)

In [31]:
batch = next(iter(train_dataloader))

In [8]:
batch['image_target'].shape

torch.Size([16, 3, 224, 224])

In [76]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)

Using cache found in /home/rliu/.cache/torch/hub/pytorch_vision_v0.10.0


In [77]:
model.layer3 = torch.nn.Sequential()
model.layer4 = torch.nn.Sequential()
model.fc = torch.nn.Sequential()
model.avgpool = torch.nn.Sequential()

In [78]:
feature = model(image_cond.cpu())

In [79]:
feature.shape

torch.Size([64, 100352])

In [48]:
proj_layer(feature.reshape([64, 128, 28, 28])).shape

torch.Size([64, 64, 14, 14])

In [38]:
proj_layer = torch.nn.Conv2d(256, 32, 1)

# model

In [ ]:
class sfm(torch.nn.Module):
    def __init__(self, resnet=True):
        super(sfm, self).__init__()
        
        if resnet:
            print('initializing resnet weights')
            self.cnn = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
            self.cnn.conv1 = torch.nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.cnn.layer3 = torch.nn.Sequential()
            self.cnn.layer4 = torch.nn.Sequential()
            self.cnn.avgpool = torch.nn.Sequential()
            self.cnn.fc = torch.nn.Sequential()
            self.proj_layer = torch.nn.Conv2d(128, 16, 1, bias=False)
            self.linear1 = torch.nn.Linear(12544, 64)
            self.activation = torch.nn.ReLU()
            self.linear2 = torch.nn.Linear(64, 4)
    
        else:
            print('initializing vit weights')
            self.cond_net = torchvision.models.vit_b_16(torchvision.models.ViT_B_16_Weights.DEFAULT)
            self.cond_net.heads.head = torch.nn.Sequential()
            self.target_net = torchvision.models.vit_b_16(torchvision.models.ViT_B_16_Weights.DEFAULT)
            self.target_net.heads.head = torch.nn.Sequential()
            self.linear1 = torch.nn.Linear(768 * 2, 768 * 2)
            self.activation = torch.nn.ReLU()
            self.linear2 = torch.nn.Linear(768 * 2, 768 * 2)
            self.activation = torch.nn.ReLU()
            self.linear3 = torch.nn.Linear(768 * 2, 4)

    def forward(self, cond, target):
        B = cond.shape[0]
        x = torch.cat([cond, target], dim=1)
        x = self.cnn(x)
        x = self.proj_layer(x.reshape([B, 128, 28, 28]))
        x = self.linear1(x.reshape(B, -1))
        x = self.activation(x)
        x = self.linear2(x)
        return x

In [94]:
model = sfm(True).to(device)

initializing resnet weights


Using cache found in /home/rliu/.cache/torch/hub/pytorch_vision_v0.10.0


In [95]:
image_target, image_cond, T = batch['image_target'].to(device), batch['image_cond'].to(device), batch['T'].to(device)

In [96]:
image_target.shape, image_cond.shape, T.shape

(torch.Size([64, 3, 224, 224]),
 torch.Size([64, 3, 224, 224]),
 torch.Size([64, 4]))

In [97]:
pred = model(image_cond, image_target)

In [98]:
loss = ((pred - T) ** 2).mean()

In [99]:
pred.shape

torch.Size([64, 4])

In [16]:
# train_dataloader = DataLoader(objaverse_sfm(dataset_root, 4, train=True, transform = ToTensor()),\
#                               batch_size=64, shuffle=True, num_workers=8)
# test_dataloader = DataLoader(objaverse_sfm(dataset_root, 4, train=False, transform = ToTensor()),\
#                              batch_size=64, shuffle=False, num_workers=8)
# azimuth = []
# sin = []
# cos = []
# for i, batch in tqdm(enumerate(train_dataloader, 0), total=50):
#     sin.append(batch['T'][:, 1])
#     cos.append(batch['T'][:, 2])
#     if i == 50:
#         break

In [17]:
# plt.hist(torch.cat(azimuth).numpy(), bins=40)

# training

## unzip files

In [18]:
# from sh import gunzip
# import json
# from tqdm.notebook import tqdm
# import os

# path = '/home/rliu/Desktop/cvfiler04/datasets/objaverse/hf-objaverse-v1/views_whole_sphere'
# with open('/home/rliu/Desktop/cvfiler04/datasets/objaverse/hf-objaverse-v1/views_whole_sphere/valid_paths.json') as f:
#     paths = json.load(f)
# total_view = 4
# gz = []
# for i, id in tqdm(enumerate(paths), total = len(paths)):
#     for filename in os.listdir(os.path.join(path, id)):
        
#         if 'gz' in filename:
# #             print(filename[:-3])
#             gz.append(id)
# #             gunzip(os.path.join(path, id, filename[:-3]))

In [100]:
device = 6

In [104]:
train_set = objaverse_sfm(dataset_root, 4, train=True, transform = ToTensor())
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=64, drop_last=False)

test_set = objaverse_sfm(dataset_root, 4, train=False, transform = ToTensor())
test_dataloader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=64, drop_last=False)

model = sfm(resnet=True).to(device)

print('\n================== total trainable parameters: %d ==================\n' % sum(p.numel() for p in model.parameters() if p.requires_grad))
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

# model = torch.nn.DataParallel(model, device_ids=[6, 7])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.1)

for epoch in range(50):  # loop over the dataset multiple times

    print('validation')
    model.eval()
    test_loss = torch.zeros(4).to(device)
    with torch.no_grad():
        for i, batch in tqdm(enumerate(test_dataloader, 0), total=len(test_dataloader)):
            # get the inputs; data is a list of [inputs, labels]
            image_target, image_cond, T_gt = \
                batch['image_target'].to(device), batch['image_cond'].to(device), batch['T'].to(device)

            # forward + backward + optimize
            T_pred = model(image_cond, image_target)
            loss = (T_pred - T_gt).abs()
            test_loss += loss.sum(dim=0)
    
    print('validation loss for epoch %d:' % epoch, test_loss.cpu() / len(test_set), test_loss.cpu().sum().item() / len(test_set))
    
    running_loss = torch.zeros(4).to(device)
    
    model.train()
    
    for i, batch in tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        image_target, image_cond, T_gt = \
            batch['image_target'].to(device), batch['image_cond'].to(device), batch['T'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        T_pred = model(image_cond, image_target)
        loss = ((T_pred - T_gt)**2).mean(dim=0)
        l1_loss = ((T_pred - T_gt).abs()).mean(dim=0)
        running_loss += l1_loss.detach()
        loss = loss.sum()
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{i-199}, {i+1:d}] loss: ', running_loss.detach().cpu() / 200, \
                  running_loss.detach().cpu().sum() / 200)
            running_loss = torch.zeros(4).to(device)

            

initializing resnet weights


Using cache found in /home/rliu/.cache/torch/hub/pytorch_vision_v0.10.0
/home/rliu/ruoshi/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



================== total trainable parameters: 7119620 ==================

cnn.conv1.weight torch.Size([64, 6, 7, 7])
cnn.bn1.weight torch.Size([64])
cnn.bn1.bias torch.Size([64])
cnn.layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
cnn.layer1.0.bn1.weight torch.Size([64])
cnn.layer1.0.bn1.bias torch.Size([64])
cnn.layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
cnn.layer1.0.bn2.weight torch.Size([64])
cnn.layer1.0.bn2.bias torch.Size([64])
cnn.layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
cnn.layer1.1.bn1.weight torch.Size([64])
cnn.layer1.1.bn1.bias torch.Size([64])
cnn.layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
cnn.layer1.1.bn2.weight torch.Size([64])
cnn.layer1.1.bn2.bias torch.Size([64])
cnn.layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
cnn.layer2.0.bn1.weight torch.Size([128])
cnn.layer2.0.bn1.bias torch.Size([128])
cnn.layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
cnn.layer2.0.bn2.weight torch.Size([128])
cnn.layer2.0.bn2.bias torch.Size([128])
cnn.layer2.0.d

  0%|          | 0/63 [00:00<?, ?it/s]

validation loss for epoch 0: tensor([0.7636, 0.6330, 0.6397, 0.2344]) 2.270752014297795


...

  0%|          | 0/63 [00:00<?, ?it/s]

validation loss for epoch 21: tensor([0.3641, 0.5249, 0.4662, 0.2204]) 1.5755831480205462


  0%|          | 0/6174 [00:00<?, ?it/s]

[0, 200] loss:  tensor([0.2886, 0.4948, 0.4413, 0.0980]) tensor(1.3228)
[200, 400] loss:  tensor([0.2925, 0.4953, 0.4435, 0.0968]) tensor(1.3281)
[400, 600] loss:  tensor([0.2959, 0.4914, 0.4455, 0.0983]) tensor(1.3312)
[600, 800] loss:  tensor([0.2928, 0.4937, 0.4446, 0.0983]) tensor(1.3294)
[800, 1000] loss:  tensor([0.2948, 0.4964, 0.4408, 0.0980]) tensor(1.3300)
[1000, 1200] loss:  tensor([0.2940, 0.4935, 0.4458, 0.0994]) tensor(1.3327)
[1200, 1400] loss:  tensor([0.2929, 0.4928, 0.4426, 0.0979]) tensor(1.3262)
[1400, 1600] loss:  tensor([0.2930, 0.4920, 0.4422, 0.0986]) tensor(1.3259)
[1600, 1800] loss:  tensor([0.2950, 0.4922, 0.4442, 0.0979]) tensor(1.3293)


KeyboardInterrupt: 

In [22]:
train_set = objaverse_sfm(dataset_root, 4, train=True, transform = ToTensor())
train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True, num_workers=32, drop_last=False)

test_set = objaverse_sfm(dataset_root, 4, train=False, transform = ToTensor())
test_dataloader = DataLoader(test_set, batch_size=64, shuffle=False, num_workers=16, drop_last=False)

model = sfm(resnet=True).to(device)
model.cnn.layer1.requires_grad_(False)
model.cnn.layer2.requires_grad_(False)
model.cnn.layer3.requires_grad_(False)

print('\n================== total trainable parameters: %d ==================\n' % sum(p.numel() for p in model.parameters() if p.requires_grad))
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

model = torch.nn.DataParallel(model, device_ids=[6, 7])
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.3)

for epoch in range(50):  # loop over the dataset multiple times

    print('validation')
    model.eval()
    test_loss = torch.zeros(4).to(device)
    with torch.no_grad():
        for i, batch in tqdm(enumerate(test_dataloader, 0), total=len(test_dataloader)):
            # get the inputs; data is a list of [inputs, labels]
            image_target, image_cond, T_gt = \
                batch['image_target'].to(device), batch['image_cond'].to(device), batch['T'].to(device)

            # forward + backward + optimize
            T_pred = model(image_cond, image_target)
            loss = (T_pred - T_gt).abs()
            test_loss += loss.sum(dim=0)
    
    print('validation loss for epoch %d:' % epoch, test_loss.cpu() / len(test_set), test_loss.cpu().sum().item() / len(test_set))
    
    running_loss = torch.zeros(4).to(device)
    
    model.train()
    
    for i, batch in tqdm(enumerate(train_dataloader, 0), total=len(train_dataloader)):
        # get the inputs; data is a list of [inputs, labels]
        image_target, image_cond, T_gt = \
            batch['image_target'].to(device), batch['image_cond'].to(device), batch['T'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        T_pred = model(image_cond, image_target)
        loss = ((T_pred - T_gt)**2).mean(dim=0)
        l1_loss = ((T_pred - T_gt).abs()).mean(dim=0)
        running_loss += l1_loss.detach()
        loss = loss.sum()
        loss.backward()
        optimizer.step()

        # print statistics
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{i-199}, {i+1:d}] loss: ', running_loss.detach().cpu() / 200, \
                  running_loss.detach().cpu().sum() / 200)
            running_loss = torch.zeros(4).to(device)

            

initializing resnet weights

================== total trainable parameters: 8677380 ==================

cnn.conv1.weight torch.Size([64, 6, 7, 7])
cnn.bn1.weight torch.Size([64])
cnn.bn1.bias torch.Size([64])
cnn.layer4.0.conv1.weight torch.Size([512, 256, 3, 3])
cnn.layer4.0.bn1.weight torch.Size([512])
cnn.layer4.0.bn1.bias torch.Size([512])
cnn.layer4.0.conv2.weight torch.Size([512, 512, 3, 3])
cnn.layer4.0.bn2.weight torch.Size([512])
cnn.layer4.0.bn2.bias torch.Size([512])
cnn.layer4.0.downsample.0.weight torch.Size([512, 256, 1, 1])
cnn.layer4.0.downsample.1.weight torch.Size([512])
cnn.layer4.0.downsample.1.bias torch.Size([512])
cnn.layer4.1.conv1.weight torch.Size([512, 512, 3, 3])
cnn.layer4.1.bn1.weight torch.Size([512])
cnn.layer4.1.bn1.bias torch.Size([512])
cnn.layer4.1.conv2.weight torch.Size([512, 512, 3, 3])
cnn.layer4.1.bn2.weight torch.Size([512])
cnn.layer4.1.bn2.bias torch.Size([512])
linear1.weight torch.Size([512, 512])
linear1.bias torch.Size([512])
linear2.weig

  0%|          | 0/125 [00:00<?, ?it/s]

validation loss for epoch 0: tensor([0.7673, 0.6362, 0.6355, 0.2494]) 2.288342618547983


  0%|          | 0/12347 [00:00<?, ?it/s]

[0, 200] loss:  tensor([0.6100, 0.6377, 0.6343, 0.1830]) tensor(2.0650)
[200, 400] loss:  tensor([0.5451, 0.6400, 0.6271, 0.1669]) tensor(1.9791)
[400, 600] loss:  tensor([0.5230, 0.6376, 0.6221, 0.1579]) tensor(1.9406)
[600, 800] loss:  tensor([0.5085, 0.6405, 0.6128, 0.1560]) tensor(1.9178)
[800, 1000] loss:  tensor([0.4991, 0.6318, 0.6167, 0.1528]) tensor(1.9004)
[1000, 1200] loss:  tensor([0.4901, 0.6396, 0.6111, 0.1493]) tensor(1.8900)
[1200, 1400] loss:  tensor([0.4919, 0.6355, 0.6108, 0.1493]) tensor(1.8875)
[1400, 1600] loss:  tensor([0.4757, 0.6398, 0.6017, 0.1456]) tensor(1.8629)
[1600, 1800] loss:  tensor([0.4697, 0.6348, 0.6084, 0.1428]) tensor(1.8557)
[1800, 2000] loss:  tensor([0.4670, 0.6384, 0.6060, 0.1430]) tensor(1.8545)
[2000, 2200] loss:  tensor([0.4684, 0.6418, 0.5965, 0.1406]) tensor(1.8472)
[2200, 2400] loss:  tensor([0.4615, 0.6384, 0.6001, 0.1400]) tensor(1.8401)
[2400, 2600] loss:  tensor([0.4582, 0.6339, 0.5986, 0.1406]) tensor(1.8313)
[2600, 2800] loss:  ten

...

validation loss for epoch 11: tensor([0.3486, 0.5551, 0.5076, 0.1156]) 1.5269081996993235


  0%|          | 0/12347 [00:00<?, ?it/s]

[0, 200] loss:  tensor([0.3336, 0.5526, 0.5030, 0.1112]) tensor(1.5004)
[200, 400] loss:  tensor([0.3440, 0.5522, 0.4977, 0.1113]) tensor(1.5052)
[400, 600] loss:  tensor([0.3454, 0.5556, 0.4965, 0.1113]) tensor(1.5088)
[600, 800] loss:  tensor([0.3363, 0.5506, 0.5017, 0.1121]) tensor(1.5006)
[800, 1000] loss:  tensor([0.3387, 0.5501, 0.4984, 0.1122]) tensor(1.4993)
[1000, 1200] loss:  tensor([0.3416, 0.5506, 0.5075, 0.1118]) tensor(1.5115)
[1200, 1400] loss:  tensor([0.3360, 0.5488, 0.5056, 0.1109]) tensor(1.5014)
[1400, 1600] loss:  tensor([0.3367, 0.5548, 0.4989, 0.1109]) tensor(1.5013)
[1600, 1800] loss:  tensor([0.3416, 0.5500, 0.5018, 0.1124]) tensor(1.5058)
[1800, 2000] loss:  tensor([0.3416, 0.5486, 0.5017, 0.1120]) tensor(1.5040)
[2000, 2200] loss:  tensor([0.3388, 0.5486, 0.5058, 0.1128]) tensor(1.5061)
[2200, 2400] loss:  tensor([0.3376, 0.5537, 0.5034, 0.1109]) tensor(1.5057)
[2400, 2600] loss:  tensor([0.3377, 0.5512, 0.5017, 0.1116]) tensor(1.5022)
[2600, 2800] loss:  ten

KeyboardInterrupt: 

Process Process-573:
Process Process-559:
Process Process-555:
Process Process-576:
Process Process-575:
Process Process-564:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rliu/ruoshi/anaconda3/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/rliu/ruoshi/anaconda3/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/rliu/ruoshi/anaconda3/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/rliu/ruoshi/anaconda3/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/rliu/ruoshi/anaconda3/lib/python3.8/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/rliu/